In [ ]:
cd '/content/drive/MyDrive/NasrAbadi/EEG Time Series /Four Sec without Overlap/With Baseline/ViT shape'

/content/drive/MyDrive/NasrAbadi/EEG Time Series /Four Sec without Overlap/With Baseline/ViT shape


In [ ]:
!pip install -U tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.1 MB/s eta 0:00:00


# Imports

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import utils as np_utils
from sklearn.model_selection import StratifiedGroupKFold, GridSearchCV
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout, multiply
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.optimizers.experimental import AdamW
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
import math
import gc
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
import tensorflow as tf
import numpy as np

# Data Loading, Preprocessing, and Splitting for EEG Signal Classification

In [ ]:
import numpy as np

# Loading EEG data
X = np.load('X.npy')  # Load feature data
Y = np.load('Y.npy')  # Load label data

# Display shapes of the data
print("X Shape:", X.shape)  # Print the shape of X
print("Y shape:", Y.shape)  # Print the shape of Y
print(Y)  # Print Y data

import scipy.io

# Loading and processing segment information
SegmentPerSubject = scipy.io.loadmat('SegmentPerSubject', mat_dtype=True)  # Load segment data
SegmentPerSubject = SegmentPerSubject['SegmentPerSubject']  # Extract segment data
SegmentPerSubject = np.array(SegmentPerSubject)  # Convert to numpy array
SegmentPerSubject = SegmentPerSubject.transpose()  # Transpose the segment data
print("shape Segments:", SegmentPerSubject.shape)  # Print the shape of segment data

# Grouping data by subject
Group = []
for i in range(SegmentPerSubject.shape[0]):
    for j in range(int(SegmentPerSubject[i])):
        Group.append(i)  # Append group information
Group = np.array(Group)  # Convert to numpy array
Group = np.expand_dims(Group, axis=1)  # Add a new dimension
print(Group)  # Print group data
print("Group shape is:", Group.shape)  # Print the shape of group data

# Data preparation for cross-validation
group_kfold1 = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=13)  # Initialize stratified k-fold
gc.collect()  # Collect garbage

# Splitting data for testing
split_test = group_kfold1.split(X, Y, Group)  # Split the data
gc.collect()  # Collect garbage

d_test = next(split_test)  # Get the first split
gc.collect()  # Collect garbage

# Creating training and testing sets
X_train = X[d_test[0]]  # Training features
gc.collect()  # Collect garbage
Y_train = Y[d_test[0]]  # Training labels
gc.collect()  # Collect garbage
X_test = X[d_test[1]]  # Testing features
gc.collect()  # Collect garbage
Y_test = Y[d_test[1]]  # Testing labels
gc.collect()  # Collect garbage
Group_train = Group[d_test[0]]  # Training groups
gc.collect()  # Collect garbage
Group_test = Group[d_test[1]]  # Testing groups
gc.collect()  # Collect garbage

# Clean up the workspace
del X, Y, d_test, split_test, Group, group_kfold1

# Reshaping data for the model
X_train = np.transpose(X_train, (0, 2, 3, 1))  # Reshape training features
X_test = np.transpose(X_test, (0, 2, 3, 1))  # Reshape testing features
print("X_train shape:", X_train.shape)  # Print shape of training features
print("X_test shape:", X_test.shape)  # Print shape of testing features

# Processing labels for the model
Y_test = np.expand_dims(Y_test, axis=1)  # Expand dimensions of testing labels
Y_train = np.expand_dims(Y_train, axis=1)  # Expand dimensions of training labels
Y_test = np_utils.to_categorical(Y_test)  # Convert testing labels to categorical
Y_train = np_utils.to_categorical(Y_train)  # Convert training labels to categorical
print('Y_test shape:', Y_test.shape)  # Print shape of testing labels
print('Y_train shape:', Y_train.shape)  # Print shape of training labels

X Shape: (4173, 1, 19, 512)
Y shape: (4173,)
[0 0 0 ... 1 1 1]
shape Segments: (121, 1)
[[  0]
 [  0]
 [  0]
 ...
 [120]
 [120]
 [120]]
Group shape is: (4173, 1)
X_train shape: (3709, 19, 512, 1)
X_test shape: (464, 19, 512, 1)
Y_test shape: (464, 2)
Y_train shape: (3709, 2)


# Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

# Configuration and Hyperparameter Settings

In [ ]:
# Setting up model parameters and configurations

learning_rate = 0.001  # Learning rate for model training
weight_decay = 0.0001  # Weight decay factor for regularization
batch_size = 256       # Batch size for model training
num_epochs = 100       # Number of epochs for training the model
time_step = 128        # Time-step dimension of the EEG data
electrodes = 19        # Number of electrodes in the EEG data
num_patches = 4        # Number of patches to divide EEG signals into
projection_dim = 10    # Dimensionality of the projection space
num_heads = 8          # Number of heads in multi-head attention layers
transformer_units = [
    projection_dim * 2,  # Double the projection dimension
    projection_dim,      # Keep the original projection dimension
]  # Size of the transformer layers
transformer_layers = 6  # Number of transformer layers in the model
mlp_head_units = [50, 10]  # Size of the dense layers in the final classifier


# Defining Custom Patch Extraction Layer

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf

# Defining a custom layer to create patches from EEG images
class Patches(layers.Layer):
    def __init__(self, electrodes, time_step):
        super().__init__()
        self.electrodes = electrodes  # Number of electrodes in the EEG data
        self.time_step = time_step    # Time-step dimension of the EEG data

    def call(self, images):
        batch_size = tf.shape(images)[0]  # Determine the batch size

        # Extract patches from the EEG images
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.electrodes, self.time_step, 1],  # Size of each patch
            strides=[1, self.electrodes, self.time_step, 1],  # Strides for patch extraction
            rates=[1, 1, 1, 1],  # Rate of extraction
            padding="VALID",     # Padding method
        )
        patch_dims = patches.shape[-1]  # Dimensions of the extracted patches
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])  # Reshape patches
        return patches


# Defining MLP (Classification head)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Defining the Multilayer Perceptron (MLP) function
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)  # Dense layer with GELU activation
        x = layers.Dropout(dropout_rate)(x)  # Dropout layer for regularization
    return x  # Return the final layer output

In [ ]:

patches = Patches(electrodes,time_step)(X_train)
print(f"Image size: {19} X {512}")
print(f"Patch size: {electrodes} X {time_step}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")


Image size: 19 X 512
Patch size: 19 X 128
Patches per image: 4
Elements per patch: 2432


# PatchEncoder Layer for Positional Encoding

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Defining the PatchEncoder layer class
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches  # Number of patches per EEG signal
        self.projection = layers.Dense(units=projection_dim)  # Dense layer for projection
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim  # Positional embedding layer
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)  # Position indices
        encoded = self.projection(patch) + self.position_embedding(positions)  # Project and add position embedding
        return encoded  # Return the encoded patches

# Building the NeuroViT

In [ ]:
from tensorflow.keras import layers, Model

num_classes = 2  # Number of classes for classification (ADHD vs Control)
input_shape = (19, 512, 1)  # Shape of the input EEG data

# Function to create the Vision Transformer (ViT) classifier
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)  # Input layer

    # Create patches from input images (EEG data)
    patches = Patches(electrodes, time_step)(inputs)

    # Encode patches with positional information
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Building the Transformer layers
    for _ in range(transformer_layers):
        # Layer normalization 1
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

        # Multi-head attention layer
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)

        # Skip connection 1
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer normalization 2
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)

        # MLP
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)

        # Skip connection 2
        encoded_patches = layers.Add()([x3, x2])

    # Creating a tensor for model representation
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)

    # Adding MLP for final classification
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.1)

    # Output layer for classification
    logits = layers.Dense(num_classes, activation="softmax")(features)

    # Creating the Keras model
    model = Model(inputs=inputs, outputs=logits)
    return model


# Training and Evaluating NeuroViT

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_addons as tfa
import keras

# Function to run the training experiment
def run_experiment(model):
    model.summary()  # Print the model summary

    # Initialize the optimizer with specified learning rate and weight decay
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    # Compile the model with binary crossentropy loss and accuracy metrics
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'],
    )

    # Set up checkpointing to save the best model
    checkpoint_filepath = "/path/to/checkpoint"  # Replace with actual path
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    # Train the model
    history = model.fit(
        x=X_train,
        y=Y_train,
        batch_size=32,
        epochs=100,
        validation_data=(X_test, Y_test),
        callbacks=[checkpoint_callback],
        verbose=2,
        shuffle=True
    )

    # Load the best weights and evaluate the model on the test set
    model.load_weights(checkpoint_filepath)
    _, accuracy = model.evaluate(X_test, Y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    return history

# Creating the ViT classifier and running the experiment
vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 19, 512, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 patches_1 (Patches)            (None, None, 2432)   0           ['input_1[0][0]']                
                                                                                                  
 patch_encoder (PatchEncoder)   (None, 4, 10)        24370       ['patches_1[0][0]']              
                                                                                                  
 layer_normalization (LayerNorm  (None, 4, 10)       20          ['patch_encoder[0][0]']      